In [ ]:
# Merging if any
import networkx as nx
import matplotlib.pyplot as plt


def plot_graph(graph, title):
    pos = nx.spring_layout(graph)
    nx.draw(graph, pos, with_labels=True, font_weight='bold')
    edge_labels = {(u, v): graph[u][v]['label'] for u, v in graph.edges()}
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_size=10)
    plt.title(title)
    plt.show()

def bounding_box_intersection_area(boxA, boxB):
    """
    Calculate the area of intersection between two bounding boxes.

    Each box is defined as [x1, y1, x2, y2] where (x1, y1) is the top-left coordinate, and (x2, y2) is the bottom-right coordinate.

    :param boxA: List of four integers for the first bounding box.
    :param boxB: List of four integers for the second bounding box.
    :return: Area of intersection or 0 if no overlap.
    """

    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Compute the width and height of the intersection rectangle
    width = max(0, xB - xA) # this would return zero if there is no overlap
    height = max(0, yB - yA) # the same

    # Compute the area of intersection
    area_intersecting = width * height
    area_box_A = abs(boxA[2] - boxA[0]) * abs(boxA[3] - boxA[1])
    area_box_B = abs(boxB[2] - boxB[0]) * abs(boxB[3] - boxB[1])
    min_AandB = min(area_box_A,area_box_B)
    percentage_overlap = area_intersecting / min_AandB  
    
    return percentage_overlap
def merge_graphlets(graphlets, bounding_boxes):
    # Find the bounding box with the largest area
    largest_area_box = max(bounding_boxes, key=lambda box: (box[2] - box[0]) * (box[3] - box[1]))

    # Corresponding graphlet to the largest area bounding box
    largest_area_graphlet = graphlets[bounding_boxes.index(largest_area_box)].copy()

    # Initialize lists to track merged and non-merged graphlets
    merged_graphlets = []
    non_merged_graphlets = []

    # Initialize variables to track the sum of nodes and edges in merged and non-merged graphlets
    merged_nodes_sum = 0
    merged_edges_sum = 0
    non_merged_nodes_sum = 0
    non_merged_edges_sum = 0

    # Merge the largest area graphlet with other graphlets
    for i, box_i in enumerate(bounding_boxes):
        # Skip the largest area bounding box
        if box_i == largest_area_box:
            continue

        # Check for significant overlap (over 50%) with the largest area bounding box
        overlap_percentage = bounding_box_intersection_area(largest_area_box, box_i)

        if overlap_percentage > 0.5:
            # Check for common nodes before merging
            common_nodes = set(graphlets[i].nodes) & set(largest_area_graphlet.nodes)

            if common_nodes:
                # Merge nodes and edges while avoiding duplications
                for node, data in graphlets[i].nodes(data=True):
                    if node not in largest_area_graphlet:
                        largest_area_graphlet.add_node(node, **data)

                for source, target ,label in graphlets[i].edges(data=True):
                
                    if (source, target) not in largest_area_graphlet.edges:
                        largest_area_graphlet.add_edge(source, target, **label)

            else:
                # Append to the non-merged graphlets list
                non_merged_graphlets.append(graphlets[i])
                # Update sums
                non_merged_nodes_sum += len(graphlets[i].nodes)
                non_merged_edges_sum += len(graphlets[i].edges)
        else:
            # Append to the non-merged graphlets list
            non_merged_graphlets.append(graphlets[i])
            # Update sums
            non_merged_nodes_sum += len(graphlets[i].nodes)
            non_merged_edges_sum += len(graphlets[i].edges)

    # Append the largest area graphlet to the merged graphlets list
    merged_graphlets.append(largest_area_graphlet)
    # Update sums
    merged_nodes_sum += len(largest_area_graphlet.nodes)
    merged_edges_sum += len(largest_area_graphlet.edges)

    # Print the sums before returning the result
    print("Merged Graphlets - Nodes:", merged_nodes_sum, "Edges:", merged_edges_sum)
    print("Non-Merged Graphlets - Nodes:", non_merged_nodes_sum, "Edges:", non_merged_edges_sum)

    for i in merged_graphlets:
        plot_graph(i,"Merged")
    for j in non_merged_graphlets:
        plot_graph(j,"Unmerged")


    return merged_graphlets, non_merged_graphlets